In [ ]:
import xgboost as xgb

print("Dask Installed ?", xgb.dask.DASK_INSTALLED)

#client = xgb.dask.Client(n_workers=4, threads_per_worker=4)

In [ ]:
from dask.distributed import Client
client = Client()  # connect to cluster

In [ ]:
client = Client(n_workers=1, threads_per_worker=4)

In [ ]:
import dask_xgboost as dxgb

In [ ]:
print("Dask Installed ?", dxgb.DASK_INSTALLED)

In [ ]:
import xgboost as xgb
xgb.__version__

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import xgboost as xgb
import sklearn
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [ ]:
boston = load_boston()

In [ ]:
boston_df = pd.DataFrame(data=boston.data, columns = boston.feature_names)
# Add one column with the target
boston_df["Price"] = boston.target
boston_df.head()

In [ ]:
# Splitting the dataset
X_train, X_test, Y_train, Y_test = train_test_split(boston.data, boston.target, train_size=0.90, random_state=42)

In [ ]:
dmat_train = xgb.DMatrix(X_train, Y_train, feature_names=boston.feature_names)
dmat_test = xgb.DMatrix(X_test, Y_test, feature_names=boston.feature_names)

In [ ]:
tweedie_booster = xgb.train({'max_depth': 3, 'eta': 1, 'objective': 'reg:tweedie',
                             'tree_method':'hist', 'nthread':4,
                             'monotone_constraints':(1,0,1,-1,1,0,1,0,-1,1,1, -1, 1)},
                    dmat_train,
                    evals=[(dmat_train, "train"), (dmat_test, "test")])

In [ ]:
print("\nTest  R2 Score : %.2f"%r2_score(Y_test, tweedie_booster.predict(dmat_test)))
print("Train R2 Score : %.2f"%r2_score(Y_train, tweedie_booster.predict(dmat_train)))

In [ ]:
from dask.distributed import Client
#When we create an instance of dask client without giving the IP address and port of scheduler, it'll create a cluster on the local machine itself.
client = Client(n_workers=1, threads_per_worker=4)

In [ ]:
client

In [ ]:
params = {'max_depth': 3, 'eta': 1, 'objective': 'reg:tweedie',
                             'tree_method':'hist', 'nthread':4,
                             'monotone_constraints':(1,0,1,-1,1,0,1,0,-1,1,1, -1, 1)}
import dask_xgboost as dxgb
bst = dxgb.train(client, params, dmat_train, Y_train)